In [31]:
%pylab inline
import json
import arrow

Populating the interactive namespace from numpy and matplotlib


In [34]:
print(arrow)

<module 'arrow' from '/Library/Python/2.7/site-packages/arrow/__init__.pyc'>


In [114]:
path = '/Users/evanzamir/Downloads/'
checks = sc.textFile("{}".format(path + 'check_*.txt'))
print(checks.first())
checks_json = (checks
              .map(lambda line: json.loads(line)['msg'])
              .map(lambda jsn: ((jsn['userid'], 
                                jsn['checkeduserid']), 1))
              .distinct()
              .cache())
print(checks_json.take(5))

{"msg":{"checkeduserid":89533778,"userid":87704905,"id":6204176352,"datetime":1435856400000},"dt":1435856403701,"rk":["check"]}
[((86938444, 53269301), 1), ((72762129, 61384378), 1), ((78105511, 53574369), 1), ((80293049, 76458212), 1), ((78409737, 72029930), 1)]


In [98]:
path = '/Users/evanzamir/Downloads/'
searches = sc.textFile("{}".format(path + 'usersearch_*.txt'))
print(searches.first())
searches_json = (searches
                  .map(lambda line: json.loads(line)['msg'])
                  .map(lambda jsn: ((jsn['userId'], 
                                    jsn['resultUserId']),
                                    jsn['resultPositionInGrid']))
                  .cache())
print(searches_json.first())

{"msg":{"resultLongitude":-66.044,"searchId":"4aef30a4-4050-42dc-8ccd-9a7a2468e42d","querySearchName":null,"queryGender":1,"querySearchlevel":6,"userId":57141829,"queryHasPicture":false,"datetime":1435856400230,"queryOnlineStatus":false,"queryOffset":0,"queryMinAge":26,"queryLatitude":18.423,"isTravelling":false,"resultUserId":66753534,"queryLimit":44,"queryMaxAge":34,"queryEthnicityMulti":null,"queryLongitude":-66.137,"resultPositionInGrid":28,"resultLatitude":18.241,"queryInterestedIn":2,"resultLastSeen":1435856321000,"resultIsOnline":true,"resultIsTravelling":false},"dt":1435856400609,"rk":["usersearch","result"]}
((57141829, 66753534), 28)


In [136]:
# search_checks_jn = searches_json.rightOuterJoin(checks_json).cache()
print(search_checks_jn.filter(lambda (key, (pos, res)): pos is not None).take(10))
impressions = (search_checks_jn
              .distinct()
              .count())
profile_views = (search_checks_jn
                .filter(lambda (key, (pos, res)): pos is not None)
                .distinct()
                .count())
print("{} impressions; {} profile views; {}% checked".format(impressions, profile_views, 100.0*profile_views/impressions))

[((84287524, 89762402), (256, 1)), ((72679795, 89445319), (10, 1)), ((88910635, 87111295), (14, 1)), ((62160625, 89572733), (18, 1)), ((71243141, 84095921), (6, 1)), ((73487082, 88565648), (2, 1)), ((87134278, 89945500), (0, 1)), ((87134278, 89945500), (1, 1)), ((87134278, 89945500), (52, 1)), ((62806787, 86759479), (46, 1))]
681944 impressions; 15943 profile views; 2.33787525075% checked


In [138]:
print(checks_json.count())
print(searches_json.count())

678080
441951


In [144]:
search_checks_jn.saveAsTextFile('/Users/evanzamir/Downloads/search_checks.txt')